In [ ]:
#!pip install python-slugify universal_pathlib

In [57]:
import time
import datetime
import urllib.request

import pandas as pd
from slugify import slugify
from upath import UPath as Path

In [80]:
files_dir = Path("/Users/ewheeler/dev/UNDatathon/data/gdelt_files/")

In [ ]:
territory_identifiers = pd.read_csv(files_dir / "country-codes/country_codes.csv")

In [44]:
df_fips = territory_identifiers[['Country (geonames)', 'fips (geonames)']].dropna()
df_fips = df_fips.rename(columns={'Country (geonames)': 'name', 'fips (geonames)': 'fips'})
df_fips

,name,fips
0,Algeria,AG
1,Egypt,EG
2,Libya,LY
3,Morocco,MO
4,Sudan,SU
...,...,...
278,Tokelau,TL
279,Tonga,TN
280,Tuvalu,TV
281,Wallis and Futuna,WF


In [47]:
countries = list(df_fips.fips.values)

In [49]:
countries[:5]

['AG', 'EG', 'LY', 'MO', 'SU']

In [7]:
#df = pd.read_csv("/Users/ewheeler/Downloads/results-20231104074458.csv")
#df = pd.read_csv("/Users/ewheeler/Downloads/results-20231104074641.csv", on_bad_lines='skip')
#df

In [92]:
#resp = requests.get("https://api.gdeltproject.org/api/v2/doc/doc?format=html&startdatetime=20200101000000&enddatetime=20231031235959&query=food%20sourcecountry:UP&mode=timelinevol&timezoom=yes")

In [ ]:
example_highcharts_export_redirect_url = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&startdatetime=20200101000000&enddatetime=20231031235959&query=food%20sourcecountry:KE&mode=timelinevol&timezoom=yes"
example_article_list_csv_url = "https://api.gdeltproject.org/api/v2/doc/doc?format=html&startdatetime=20200101000000&enddatetime=20231031235959&query=food%20sourcecountry:AL&mode=artlist&maxrecords=75&format=csv&sort=hybridrel"

In [165]:
%%time
query = "food"
modes = ["timelinevolraw", "timelinetone", "timelinevol"]
#start_dt = "20200101000000"
#end_dt = "20231031235959"

start_dt = "20190101000000"
end_dt = "20191231235959"
year_range = "2019"

#for fips in countries:
for fips in ['UP']:#, 'DJ', 'EG', 'KE', 'MZ', 'SU', 'TZ', 'RS']:
    for mode in modes:
        source = f"https://api.gdeltproject.org/api/v2/doc/doc?format=csv&startdatetime={start_dt}&enddatetime={end_dt}&query={query.replace(' ', '%20')}%20sourcecountry:{fips}&mode={mode}&timezoom=yes"
        destination = files_dir / f"{query}/{fips}_{slugify(query)}_{mode}_{year_range}.csv"
        if not destination.exists():
            local_filename, headers = urllib.request.urlretrieve(source, destination)
            print(local_filename)
            # endpoint is rate limited.
            # error msg says to keep big queries to
            # one per 5s, so sleep after each call
            time.sleep(6)
    source = f"https://api.gdeltproject.org/api/v2/doc/doc?format=html&startdatetime={start_dt}&enddatetime={end_dt}&query={query.replace(' ', '%20')}%20sourcecountry:{fips}&mode=artlist&maxrecords=75&format=csv&sort=hybridrel"
    destination = files_dir / f"{query}/{fips}_{slugify(query)}_artlist_{year_range}.csv"
    if not destination.exists():
        local_filename, headers = urllib.request.urlretrieve(source, destination)
        print(local_filename)
        # endpoint is rate limited.
        # error msg says to keep big queries to
        # one per 5s, so sleep after each call
        time.sleep(6)

/Users/ewheeler/dev/UNDatathon/data/gdelt_files/food/UP_food_artlist_2019.csv
CPU times: user 24.1 ms, sys: 7.44 ms, total: 31.6 ms
Wall time: 7.73 s


In [153]:
#dfs = {'artlist': list(), 'timelinetone': list(),
dfs = {'artlist': list(), 'timelinetone': list(),
       'timelinevolraw': list(), 'timelinevol': list(),
       'volumeintensity': list()}
for f in Path(files_dir / "food").glob("*.csv"):
    try:
        fips, query, api_mode, year = f.stem.split('_')
    except ValueError:
        fips, query, api_mode = f.stem.split('_')
        if '2019' in api_mode:
            continue
    # XXX change for desired query
    if query == 'food':
        print(f.stem)
        try:
            df = pd.read_csv(f)
        except pd.errors.EmptyDataError as e:
            print(f.stem, e)
            continue
        df['fips'] = fips
        df['query'] = query
        df['api_mode'] = api_mode
        dfs_list = dfs[api_mode]
        dfs_list.append(df)
        dfs.update({api_mode: dfs_list})

ES_food_artlist
AY_food_volumeintensity
SF_food_timelinevol_2019
HO_food_volumeintensity
KU_food_artlist
BY_food_timelinevolraw_2019
MF_food_volumeintensity
CK_food_timelinevolraw
CV_food_timelinetone
EC_food_timelinevol_2019
BV_food_timelinevolraw
BV_food_timelinevolraw No columns to parse from file
BD_food_timelinevol_2019
LY_food_timelinetone_2019
TZ_food_artlist
LT_food_volumeintensity
MD_food_artlist
TN_food_timelinetone
CB_food_artlist
KZ_food_timelinevolraw
DJ_food_timelinevolraw
SG_food_volumeintensity
SF_food_timelinetone
JM_food_timelinetone
DJ_food_timelinetone_2019
HO_food_artlist
MF_food_timelinevolraw
EG_food_artlist
CN_food_timelinevolraw_2019
FI_food_artlist
CV_food_artlist
BP_food_timelinetone
NH_food_timelinevolraw
MP_food_artlist
IV_food_timelinetone
TN_food_artlist
PM_food_timelinevolraw_2019
MT_food_timelinetone
TN_food_timelinevolraw
NE_food_volumeintensity
GK_food_timelinevolraw
KZ_food_timelinetone
MB_food_volumeintensity
RQ_food_timelinetone
LT_food_timelinevol

In [154]:
dfs.keys()

dict_keys(['artlist', 'timelinetone', 'timelinevolraw', 'timelinevol', 'volumeintensity'])

In [155]:
vol = dfs['timelinevol']

In [156]:
len(vol)

110

In [157]:
inte = dfs['volumeintensity']
len(inte)

222

In [161]:
dfs.update({'timelinevol': vol + inte})
dfs.pop('volumeintensity')

[            Date            Series  Value fips query         api_mode
 0     2020-01-01  Volume Intensity    0.0   AY  food  volumeintensity
 1     2020-01-02  Volume Intensity    0.0   AY  food  volumeintensity
 2     2020-01-03  Volume Intensity    0.0   AY  food  volumeintensity
 3     2020-01-04  Volume Intensity    0.0   AY  food  volumeintensity
 4     2020-01-05  Volume Intensity    0.0   AY  food  volumeintensity
 ...          ...               ...    ...  ...   ...              ...
 1393  2023-10-27  Volume Intensity    0.0   AY  food  volumeintensity
 1394  2023-10-28  Volume Intensity    0.0   AY  food  volumeintensity
 1395  2023-10-29  Volume Intensity    0.0   AY  food  volumeintensity
 1396  2023-10-30  Volume Intensity    0.0   AY  food  volumeintensity
 1397  2023-10-31  Volume Intensity    0.0   AY  food  volumeintensity
 
 [1398 rows x 6 columns],
             Date            Series   Value fips query         api_mode
 0     2020-01-01  Volume Intensity  0.0019   HO

In [163]:
for api_mode, dfs_list in dfs.items():
    if len(dfs_list) == 0:
        continue
    df = pd.concat(dfs_list)
    try:
        df = df.drop(columns=['Invalid/Unsupported Country.'])
    except KeyError:
        pass
    df['api_mode'] = api_mode
    # XXX change for query used in previous cell!
    print(api_mode, len(dfs_list), df.shape)
    df.to_feather(files_dir / f'gdelt_food_{api_mode}.feather')

artlist 211 (12816, 7)
timelinetone 332 (345217, 6)
timelinevolraw 332 (690434, 6)


/opt/homebrew/Caskroom/miniforge/base/envs/magasin/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/opt/homebrew/Caskroom/miniforge/base/envs/magasin/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/opt/homebrew/Caskroom/miniforge/base/envs/magasin/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


timelinevol 332 (345217, 6)


/opt/homebrew/Caskroom/miniforge/base/envs/magasin/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [118]:
"""
import os

for f in files_dir.glob('*.csv'):
    print(f)
    try:
        fips, query, api_mode, dt = f.stem.split('_')
    except ValueError:
        continue
    new_stem = "_".join([fips, query, api_mode])
    p = Path(str(f).replace(f.stem, new_stem))
    print(p)
    os.rename(f, p)
"""

'\nimport os\n\nfor f in files_dir.glob(\'*.csv\'):\n    print(f)\n    try:\n        fips, query, api_mode, dt = f.stem.split(\'_\')\n    except ValueError:\n        continue\n    new_stem = "_".join([fips, query, api_mode])\n    p = Path(str(f).replace(f.stem, new_stem))\n    print(p)\n    os.rename(f, p)\n'